# 머신러닝 돌리기 (기본)
### history-stations
- 새로운 변수 추가, 튜닝 등 진행하지 않음

In [ ]:
!pip install pymongo

#### 라이브러리 준비

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from prophet import Prophet
import holidays

from datetime import datetime, timedelta
now = datetime.now()

#### 방문자수 데이터 데이터 프레임 변환

In [ ]:
from pymongo import MongoClient

client = MongoClient('주소 작성')

db = client.crawling

collection = db['주소 작성']

rows = collection.find()
history_stations = []
for row in rows:
    history_stations.append(row)

history_stations = pd.DataFrame(history_stations)

#### 머신러닝 코드 RandomForestRegressor

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = RandomForestRegressor(random_state=0)

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    # print(pre_df)
    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)
    # print(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

- 5/23에 새로 생겨난 충전소에 대해서 데이터가 없어서 모델을 돌릴 수 없음 => 시간 경과 자동 해결

- blue = test, orange = predict

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

#### 머신러닝 코드 LinearRegression

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = LinearRegression()

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

- 5/23에 새로 생겨난 충전소에 대해서 데이터가 없어서 모델을 돌릴 수 없음 => 시간 경과 자동 해결

- blue = test, orange = predict

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

#### 머신러닝 코드 LGBMRegressor

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = LGBMRegressor(random_state=0, verbose=-1)

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

- 5/23에 새로 생겨난 충전소에 대해서 데이터가 없어서 모델을 돌릴 수 없음 => 시간 경과 자동 해결

- blue = test, orange = predict

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

#### 머신러닝 코드 xgboost

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = xgb.XGBRegressor(
        eval_metric='rmse'  # 회귀에서는 r2를 사용하는 경우
    )

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

# 머신러닝 성능 테스트 ⭐
(초기 0.65)
## 평가 기준 r2(결정계수)
## 1. RandomForestRegressor 👍👍
## 결과: 0.7486
## 2. LinearRegression
## 결과: 0.2363
## 3. LGBMRegressor
## 결과: 0.7023
## 4. xgboost
## 결과: 0.6801